# Esse primeiro bloco faz a instalação das bibliotecas necessárias para o treinamento de máquina e deve ser o primeiro a ser clicado.

In [ ]:
# Importação das bibliotecas necessárias
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
import transformers
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTETomek
from collections import Counter
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, LSTM, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix, classification_report
import tensorflow.keras.backend as K
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks





In [ ]:
# 📌 **Carregar os dados**
caminho_arquivo = "/content/drive/MyDrive/Treinamento_maquina/aconamentos_2023_2024.xlsx"
df = pd.read_excel(caminho_arquivo)

# 📌 **Verificação inicial**
print("📊 Dataset original:")
print(df.head())
print(df.info())

# 📌 **Correção de valores negativos no TEMPO_ATRASO**
df.loc[df['TEMPO_ATRASO'] < 0, 'TEMPO_ATRASO'] = df['TEMPO_ATRASO'].median()

# 📌 **Conversão da Data**
df['DATA'] = pd.to_datetime(df['Data'], errors='coerce')
df['MES'] = df['DATA'].dt.month  # Extrair o mês
df['BIMESTRE'] = (df['DATA'].dt.month - 1) // 2 + 1  # Criar bimestres (1-6)
df['SEMESTRE'] = (df['DATA'].dt.month - 1) // 6 + 1  # Criar semestres (1-2)

# 📌 **Conversão da Hora** (Corrigida para evitar remoção de todas as linhas)
df['Hora'] = pd.to_datetime(df['Hora'], format='%H:%M:%S', errors='coerce')
df['HORA_DECIMAL'] = df['Hora'].dt.hour + df['Hora'].dt.minute / 60

# 📌 **Verificação após conversão da Hora**
if df['HORA_DECIMAL'].isna().sum() == len(df):
    print("\n🚨 ERRO: Todas as horas falharam na conversão! Verifique o formato da coluna `Hora`.")
    print(df[['Hora']].head())
    exit()

# 📌 **Representação cíclica da Hora**
df['HORA_SIN'] = np.sin(2 * np.pi * df['HORA_DECIMAL'] / 24)
df['HORA_COS'] = np.cos(2 * np.pi * df['HORA_DECIMAL'] / 24)

# 📌 **Criar novas variáveis**
df['QTD_ACIONAMENTOS'] = df.groupby('COD')['COD'].transform('count')
df['QTD_ACORDOS'] = df.groupby('COD')['ACAO'].transform(lambda x: (x.isin(['ACD', 'ACP'])).sum())
df['ACAO_ANTERIOR'] = df.groupby('COD')['ACAO'].shift(1)
df['DIAS_ENTRE_ACIONAMENTOS'] = df.groupby('COD')['DATA'].diff().dt.days
df['MEDIA_DIAS_ACIONAMENTO'] = df.groupby('COD')['DIAS_ENTRE_ACIONAMENTOS'].transform('mean')

# 📌 **Criando Faixa de Valor da Dívida**
bins = [0, 10000, 50000, 100000, 500000, np.inf]
labels = ['Até 10k', 'Até 50k', 'Até 100k', 'Até 500k', 'Acima 500k']
df['FAIXA_VALOR_DIVIDA'] = pd.cut(df['VALOR'], bins=bins, labels=labels)

# 📌 **Número de acionamentos nos últimos 30 dias** (corrigido para evitar exclusão de linhas)
df['ULTIMOS_30_DIAS'] = df.groupby('COD')['DATA'].transform(lambda x: x.diff().dt.days.fillna(999).le(30).sum())

# 📌 **Prescrição da Dívida**
df['PRESCRITA'] = (df['TEMPO_ATRASO'] > 1825).astype(int)

# 📌 **Tratar valores ausentes**
df.fillna({'ACAO_ANTERIOR': 'SEM_ACAO',
           'DIAS_ENTRE_ACIONAMENTOS': df['DIAS_ENTRE_ACIONAMENTOS'].median(),
           'MEDIA_DIAS_ACIONAMENTO': df['MEDIA_DIAS_ACIONAMENTO'].median()}, inplace=True)

# 🔹 **Verificação de `ACAO` antes da conversão**
print("\n📊 Valores únicos em `ACAO` antes de transformar `y`:")
print(df['ACAO'].value_counts())

# 🔹 **Encoding de variáveis categóricas**
label_encoder = LabelEncoder()
df['CREDOR'] = label_encoder.fit_transform(df['CREDOR'])
df['TIPO_PESSOA'] = label_encoder.fit_transform(df['TIPO_PESSOA'])
df['ACAO_ANTERIOR'] = label_encoder.fit_transform(df['ACAO_ANTERIOR'])
df['FAIXA_VALOR_DIVIDA'] = label_encoder.fit_transform(df['FAIXA_VALOR_DIVIDA'])

# 📌 **Selecionando Features**
features = ['CREDOR', 'TIPO_PESSOA', 'TEMPO_ATRASO', 'VALOR', 'QTD_ACIONAMENTOS', 'QTD_ACORDOS',
            'MEDIA_DIAS_ACIONAMENTO', 'PRESCRITA', 'MES', 'BIMESTRE', 'SEMESTRE',
            'FAIXA_VALOR_DIVIDA', 'HORA_SIN', 'HORA_COS', 'ULTIMOS_30_DIAS']

X = df[features]
y = df['ACAO'].apply(lambda x: 1 if x in ['ACD', 'ACP'] else 0)

# 📊 **Verificação do Dataset antes da divisão**
print(f"📊 Tamanho do Dataset: {df.shape[0]} linhas")
print(f"Tamanho de X: {X.shape}")
print(f"Tamanho de y: {y.shape}")
print(f"Valores únicos em `y`: {y.value_counts()}")

# 🚨 **Se o DataFrame estiver vazio, parar a execução antes da divisão Treino/Teste**
if X.shape[0] == 0 or y.shape[0] == 0:
    print("\n🚨 ERRO: O DataFrame está vazio! Verifique os dados antes da divisão Treino/Teste.")
    exit()


# 📌 Aplicação do SMOTE com menos oversampling (30% da classe majoritária)
smote = SMOTE(sampling_strategy=0.3, k_neighbors=3, random_state=42)

# 📌 Aplicação do TomekLinks para undersampling (remover ruídos da classe majoritária)
tomek = TomekLinks(sampling_strategy='majority')

# 📌 Divisão Treino/Teste (antes do SMOTE)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# 📌 Aplicando o SMOTE primeiro para gerar exemplos sintéticos da classe minoritária
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# 📌 Aplicando o TomekLinks para remover ruídos e exemplos muito próximos da classe majoritária
X_train_resampled, y_train_resampled = tomek.fit_resample(X_train_smote, y_train_smote)

# 🔹 Normalização dos dados após o resampling
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)

# 📊 Exibir o resultado final do balanceamento
print(f"📊 Após SMOTE-Tomek: Classe 0: {sum(y_train_resampled == 0)}, Classe 1: {sum(y_train_resampled == 1)}")



def focal_loss(alpha=0.25, gamma=2.0):
    def loss(y_true, y_pred):
        bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        focal_loss = alpha * K.pow((1 - p_t), gamma) * bce
        return K.mean(focal_loss)
    return loss


In [ ]:
# ======= 📌 MLP - Multilayer Perceptron =======
mlp_model = Sequential([
    Input(shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

mlp_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callback Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)









In [ ]:
# ======= 📌 LSTM =======
X_train_seq = np.reshape(X_train_scaled, (X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_seq = np.reshape(X_test_scaled, (X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

lstm_model = Sequential([
    Input(shape=(1, X_train_scaled.shape[1])),
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# ======= 📌 Transformer =======
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.norm1 = LayerNormalization(epsilon=1e-6)
        self.norm2 = LayerNormalization(epsilon=1e-6)
        self.dense = Dense(embed_dim, activation='relu')

    def call(self, inputs):
        attn_output = self.att(inputs, inputs)
        out1 = self.norm1(inputs + attn_output)
        dense_output = self.dense(out1)
        return self.norm2(out1 + dense_output)

transformer_model = Sequential([
    Input(shape=(1, X_train_scaled.shape[1])),
    TransformerBlock(embed_dim=X_train_scaled.shape[1], num_heads=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

transformer_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])




In [ ]:
history_mlp = mlp_model.fit(X_train_scaled, y_train_resampled,
                            epochs=100, batch_size=32,
                            validation_data=(X_test_scaled, y_test),
                            callbacks=[early_stopping])

history_lstm = lstm_model.fit(X_train_seq, y_train_resampled,
                              epochs=100, batch_size=32,
                              validation_data=(X_test_seq, y_test),
                              callbacks=[early_stopping])

history_transformer = transformer_model.fit(X_train_seq, y_train_resampled,
                                            epochs=100, batch_size=32,
                                            validation_data=(X_test_seq, y_test),
                                            callbacks=[early_stopping])


In [ ]:


# 🔹 Função para Plotar Comparação do Treinamento
def plot_training_comparison(history_mlp, history_lstm, history_transformer):
    plt.figure(figsize=(14, 6))

    # 🔹 Comparação da Perda (Loss)
    plt.subplot(1, 2, 1)
    plt.plot(history_mlp.history['loss'], label='MLP - Loss')
    plt.plot(history_mlp.history['val_loss'], label='MLP - Val Loss', linestyle="dashed")

    plt.plot(history_lstm.history['loss'], label='LSTM - Loss')
    plt.plot(history_lstm.history['val_loss'], label='LSTM - Val Loss', linestyle="dashed")

    plt.plot(history_transformer.history['loss'], label='Transformer - Loss')
    plt.plot(history_transformer.history['val_loss'], label='Transformer - Val Loss', linestyle="dashed")

    plt.title('Comparação da Perda (Loss)')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # 🔹 Comparação da Acurácia
    plt.subplot(1, 2, 2)
    plt.plot(history_mlp.history['accuracy'], label='MLP - Acc')
    plt.plot(history_mlp.history['val_accuracy'], label='MLP - Val Acc', linestyle="dashed")

    plt.plot(history_lstm.history['accuracy'], label='LSTM - Acc')
    plt.plot(history_lstm.history['val_accuracy'], label='LSTM - Val Acc', linestyle="dashed")

    plt.plot(history_transformer.history['accuracy'], label='Transformer - Acc')
    plt.plot(history_transformer.history['val_accuracy'], label='Transformer - Val Acc', linestyle="dashed")

    plt.title('Comparação da Acurácia')
    plt.xlabel('Epochs')
    plt.ylabel('Acurácia')
    plt.legend()

    plt.show()

# 🔹 Chamando a função para plotar os gráficos comparativos
plot_training_comparison(history_mlp, history_lstm, history_transformer)


In [ ]:
plot_training_comparison(history_mlp, history_lstm, history_transformer)


In [ ]:

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix, classification_report


# ======= 📌 Avaliação =======
models = {'MLP': mlp_model, 'LSTM': lstm_model, 'Transformer': transformer_model}
thresholds = [0.15, 0.25]  # Testando diferentes thresholds

for threshold in thresholds:
    print(f"\n🔹 Testando Threshold: {threshold}")

    for model_name, model in models.items():
        # Obtém as probabilidades de predição
        y_pred_probs = model.predict(X_test_scaled if model_name == 'MLP' else X_test_seq)
        y_pred = (y_pred_probs > threshold).astype("int32").flatten()  # Aplica o threshold

        # Calcula métricas de avaliação
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, zero_division=1)
        recall = recall_score(y_test, y_pred, zero_division=1)
        precision = precision_score(y_test, y_pred, zero_division=1)

        # Exibe os resultados
        print(f"\n📊 {model_name} - Threshold {threshold}:")
        print(f"🔹 Acurácia: {acc:.4f} | F1-Score: {f1:.4f} | Recall: {recall:.4f} | Precisão: {precision:.4f}")
        print(f"🔹 Matriz de Confusão:\n{confusion_matrix(y_test, y_pred)}")
        print(f"\n📌 Relatório de Classificação:\n{classification_report(y_test, y_pred, zero_division=1)}")

